In [323]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score,confusion_matrix



In [324]:
BASE_DIR = Path.cwd()          # ml/
DATA_DIR = BASE_DIR.parent / "data"

def load_csv(name):
    return pd.read_csv(DATA_DIR / f"{name}.csv")


Data

In [325]:
orders=load_csv("order_level")
fact=load_csv("fact")
products=load_csv("dim_products")

In [326]:
fact['seller_id'].describe()

count                               109132
unique                                2962
top       6560211a19b47992c3666cc44a7e94c0
freq                                  1978
Name: seller_id, dtype: object

In [358]:
fact=fact.merge(products[['product_id','product_category_name']],on='product_id',how='left')
fact.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_item_id,product_id,seller_id,price,delivery_cost,review_score,review_creation_date,delivery_time,has_review,super_late,high_price,high_delivery_cost,product_category_name_x,product_category_name_y,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-20 23:43:48,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,5,2017-09-21 00:00:00,7.61,True,False,False,False,cool stuff,cool stuff,cool stuff
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-05-12 16:04:24,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,4,2017-05-13 00:00:00,16.22,True,False,True,False,pet shop,pet shop,pet shop
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-22 13:19:16,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,5,2018-01-23 00:00:00,7.95,True,False,True,False,Furniture,Furniture,Furniture
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-14 13:32:39,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,4,2018-08-15 00:00:00,6.15,True,False,False,False,perfumery,perfumery,perfumery
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-03-01 16:42:31,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,5,2017-03-02 00:00:00,25.11,True,False,True,False,garden tools,garden tools,garden tools


In [328]:
fact['order_id'].describe()

count                               109132
unique                               95371
top       5a3b1c29a49756e75f1ef513383c0c12
freq                                    22
Name: order_id, dtype: object

In [329]:
fact=fact.merge(products[['product_id','product_category_name']],on="product_id")

In [330]:
fact.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_item_id,product_id,seller_id,price,delivery_cost,review_score,review_creation_date,delivery_time,has_review,super_late,high_price,high_delivery_cost,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-20 23:43:48,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,5,2017-09-21 00:00:00,7.61,True,False,False,False,cool stuff
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-05-12 16:04:24,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,4,2017-05-13 00:00:00,16.22,True,False,True,False,pet shop
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-22 13:19:16,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,5,2018-01-23 00:00:00,7.95,True,False,True,False,Furniture
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-14 13:32:39,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,4,2018-08-15 00:00:00,6.15,True,False,False,False,perfumery
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-03-01 16:42:31,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,5,2017-03-02 00:00:00,25.11,True,False,True,False,garden tools


In [365]:
fact.groupby(['order_id','seller_id'])['delivery_time'].nunique().max()


np.int64(1)

In [368]:
new=fact.groupby(['order_id','seller_id']).agg({
    'price':'sum',
    'delivery_cost':'sum',
    'product_category_name':'first',
    'order_purchase_timestamp':'first',
    'delivery_time':'first'
})
new=new.reset_index()
new

,order_id,seller_id,price,delivery_cost,product_category_name,order_purchase_timestamp,delivery_time
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,58.90,13.29,cool stuff,2017-09-13 08:59:02,7.61
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,pet shop,2017-04-26 10:53:06,16.22
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,199.00,17.87,Furniture,2018-01-14 14:33:31,7.95
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,perfumery,2018-08-08 10:00:35,6.15
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,199.90,18.14,garden tools,2017-02-04 13:57:51,25.11
...,...,...,...,...,...,...,...
95333,fffc94f6ce00a00581880bf54a75a037,b8bc237ba3788b23da09c0f1f3a3288c,299.99,43.41,housewares,2018-04-23 13:57:06,17.37
95334,fffcd46ef2263f404302a634eb57f7eb,f3c38ab652836d21de61fb8314b69182,350.00,36.53,computers accessories,2018-07-14 10:26:46,9.42
95335,fffce4705a9662cd70adb13d4a31832d,c3cfdc648177fdbbbb35635a37472c53,99.90,16.95,sports leisure,2017-10-23 17:07:56,4.80
95336,fffe18544ffabc95dfada21779c9644f,2b3e4a2a3ea8e01938cabda2a3e5cc79,55.99,8.72,computers accessories,2017-08-14 23:02:59,1.96


In [360]:
new.duplicated(['order_id','seller_id']).any()


np.False_

In [361]:

new['seller_id'].describe()

count                                95338
unique                                2906
top       6560211a19b47992c3666cc44a7e94c0
freq                                  1801
Name: seller_id, dtype: object

In [374]:
new.duplicated(['seller_id','order_id']).any()

np.False_

In [369]:
#new['is_late']=np.where(new['delivery_time']>15,1,0)
#new['is_late']=new['delivery_time'].map(lambda x : 1 if x >15 else 0)
new['is_late']=(new['delivery_time']>15).astype(int)

In [370]:
new.head()

,order_id,seller_id,price,delivery_cost,product_category_name,order_purchase_timestamp,delivery_time,is_late
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,58.90,13.29,cool stuff,2017-09-13 08:59:02,7.61,0
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,pet shop,2017-04-26 10:53:06,16.22,1
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,199.00,17.87,Furniture,2018-01-14 14:33:31,7.95,0
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,perfumery,2018-08-08 10:00:35,6.15,0
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,199.90,18.14,garden tools,2017-02-04 13:57:51,25.11,1


In [371]:
# check the propostions of late orders
new['is_late'].value_counts(normalize=True)*100


is_late
0    73.0223
1    26.9777
Name: proportion, dtype: float64

In [372]:
new['product_category_name'].value_counts()

product_category_name
bed bath table                                   9258
health beauty                                    8580
Furniture                                        8042
sports leisure                                   7487
computers accessories                            6525
housewares                                       5733
watches gifts                                    5479
telephony                                        4087
auto                                             3806
toys                                             3766
cool stuff                                       3527
garden tools                                     3416
perfumery                                        3084
baby                                             2787
electronics                                      2501
Fashion                                          2336
stationery                                       2253
pet shop                                         1702
luggag

In [388]:
# consider only top categories and set others are 'others'

top_categories=new['product_category_name'].value_counts().nlargest(18).index

new['product_category_name']=new['product_category_name'].where(
    new['product_category_name'].isin(top_categories),
    other='others'
)

#check
new[new['product_category_name']=='others'].sort_values('delivery_time',ascending=False)

new.head()

KeyError: 'product_category_name'

In [ ]:
new=new.drop(columns=['delivery_time'])

In [ ]:
encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(new[['product_category_name']]) 

In [ ]:
encoded_df=pd.DataFrame(
    encoded,
    columns=encoder.get_feature_names_out(['product_category_name']),
    index=new.index
).astype(int)

In [ ]:
new=new.drop(columns=['product_category_name']).join(encoded_df)

In [ ]:
new['order_purchase_timestamp']=pd.to_datetime(new['order_purchase_timestamp'])


In [ ]:
new

,order_id,seller_id,price,delivery_cost,order_purchase_timestamp,is_late,product_category_name_Fashion,product_category_name_Furniture,product_category_name_auto,product_category_name_baby,product_category_name_bed bath table,product_category_name_computers accessories,product_category_name_cool stuff,product_category_name_electronics,product_category_name_garden tools,product_category_name_health beauty,product_category_name_housewares,product_category_name_others,product_category_name_perfumery,product_category_name_sports leisure,product_category_name_stationery,product_category_name_telephony,product_category_name_toys,product_category_name_watches gifts
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,58.90,13.29,2017-09-13 08:59:02,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,2017-04-26 10:53:06,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,199.00,17.87,2018-01-14 14:33:31,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,2018-08-08 10:00:35,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,199.90,18.14,2017-02-04 13:57:51,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95333,fffc94f6ce00a00581880bf54a75a037,b8bc237ba3788b23da09c0f1f3a3288c,299.99,43.41,2018-04-23 13:57:06,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
95334,fffcd46ef2263f404302a634eb57f7eb,f3c38ab652836d21de61fb8314b69182,350.00,36.53,2018-07-14 10:26:46,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
95335,fffce4705a9662cd70adb13d4a31832d,c3cfdc648177fdbbbb35635a37472c53,99.90,16.95,2017-10-23 17:07:56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
95336,fffe18544ffabc95dfada21779c9644f,2b3e4a2a3ea8e01938cabda2a3e5cc79,55.99,8.72,2017-08-14 23:02:59,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
x=new.drop(columns=['order_id','seller_id','is_late','order_purchase_timestamp'])
y=new['is_late']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=67,stratify=y)

#scaling delivery cost and price values 
columns_to_scale = ['delivery_cost','price']

scaler = StandardScaler()
x_train[columns_to_scale]=scaler.fit_transform(x_train[columns_to_scale])

x_test[columns_to_scale]=scaler.transform(x_test[columns_to_scale])


In [ ]:
y_train

78019    1
1766     0
40861    0
54954    0
810      1
        ..
22153    0
2213     1
50269    0
34861    0
47756    0
Name: is_late, Length: 76270, dtype: int64

In [ ]:


model = LogisticRegression()
model.fit(x_train,y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [ ]:
y_pred= model.predict(x_test)
y_pred
y_prob = model.predict_proba(x_test)[:, 1]  # probability of class 1


In [ ]:
x_test

,price,delivery_cost,product_category_name_Fashion,product_category_name_Furniture,product_category_name_auto,product_category_name_baby,product_category_name_bed bath table,product_category_name_computers accessories,product_category_name_cool stuff,product_category_name_electronics,product_category_name_garden tools,product_category_name_health beauty,product_category_name_housewares,product_category_name_others,product_category_name_perfumery,product_category_name_pet shop,product_category_name_sports leisure,product_category_name_stationery,product_category_name_telephony,product_category_name_toys,product_category_name_watches gifts
34062,-0.055374,-0.534851,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
52095,-0.235823,0.115190,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4300,-0.094602,0.745340,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
73136,2.925962,8.189700,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2286,0.454591,-0.030475,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43691,0.046619,-0.058068,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
10605,-0.404543,0.228771,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3594,1.164581,0.498285,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
95225,-0.287212,-0.296781,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [389]:

cm=confusion_matrix(y_test,y_pred)

acc = accuracy_score(y_test,y_pred)

acc

0.7281833438221104

0.7313197026022304


In [390]:
roc= roc_auc_score(y_test,y_prob)
print(f"accuracy score :{acc:.4f} \nroc score: {roc:.4f}\nconfusion matrix:\n{cm}")

accuracy score :0.7282 
roc score: 0.6324
confusion matrix:
[[13702   222]
 [ 4961   183]]


accuracy score :0.7313 
roc score: 0.6471
confusion matrix:
[[15378   361]
 [ 5421   360]]


In [393]:
new['seller_id'].describe()

count                                95338
unique                                2906
top       6560211a19b47992c3666cc44a7e94c0
freq                                  1801
Name: seller_id, dtype: object

In [396]:
new.duplicated(['order_id','seller_id']).any()


np.False_